# Assignment 0

In [30]:
import cv2
import numpy as np
import os

### Q1.

In [31]:
def video_to_images(video_path, output_folder):

    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    success, frame = cap.read()
    count = 0

    while success:
        image_path = os.path.join(output_folder, f"frame_{count:04d}.jpg")
        cv2.imwrite(image_path, frame)
        success, frame = cap.read()
        count += 1

    cap.release()


In [32]:
def images_to_video(input_folder, output_video_path, frame_rate):

    os.makedirs(os.path.dirname(output_video_path), exist_ok=True)

    images = [img for img in os.listdir(input_folder) if img.endswith(".jpg")]
    images.sort()

    frame = cv2.imread(os.path.join(input_folder, images[0]))
    height, width, _ = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    [video.write(cv2.imread(os.path.join(input_folder, image))) for image in images]

    video.release()

In [33]:
video_path = "earth.mp4"
pic_folder = "output_images"
output_video_path = "video/q1/new.mp4"


In [34]:
video_to_images(video_path, pic_folder)


In [35]:
images_to_video(pic_folder, output_video_path, frame_rate=30)

### Q2.

In [36]:
def save_image(frame, output_folder, count):
    image_path = os.path.join(output_folder, f"frame_{count:04d}.jpg")
    cv2.imwrite(image_path, frame)
    print(f"Saved: {image_path}")

In [37]:

def capture_images_from_webcam(output_folder, exit_key='q', save_key='s'):
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Failed to open the webcam")
        return

    count = 0
    while True:
        ret, frame = cap.read()

        cv2.imshow("Webcam", frame)

        save_image(frame, output_folder, count)

        key = cv2.waitKey(1) & 0xFF
        if key == ord(exit_key):
            break
        elif key == ord(save_key):
            save_image(frame, output_folder, count)

        count += 1

    cap.release()
    cv2.destroyAllWindows()


In [38]:

output_folder = "pics/q2/"


In [39]:
capture_images_from_webcam(output_folder)

Saved: pics/q2/frame_0000.jpg
Saved: pics/q2/frame_0001.jpg
Saved: pics/q2/frame_0002.jpg
Saved: pics/q2/frame_0003.jpg
Saved: pics/q2/frame_0004.jpg
Saved: pics/q2/frame_0005.jpg
Saved: pics/q2/frame_0006.jpg
Saved: pics/q2/frame_0007.jpg
Saved: pics/q2/frame_0008.jpg
Saved: pics/q2/frame_0009.jpg
Saved: pics/q2/frame_0010.jpg
Saved: pics/q2/frame_0011.jpg
Saved: pics/q2/frame_0012.jpg
Saved: pics/q2/frame_0013.jpg
Saved: pics/q2/frame_0014.jpg
Saved: pics/q2/frame_0015.jpg
Saved: pics/q2/frame_0016.jpg
Saved: pics/q2/frame_0017.jpg
Saved: pics/q2/frame_0018.jpg
Saved: pics/q2/frame_0019.jpg
Saved: pics/q2/frame_0020.jpg
Saved: pics/q2/frame_0021.jpg
Saved: pics/q2/frame_0022.jpg
Saved: pics/q2/frame_0023.jpg
Saved: pics/q2/frame_0024.jpg
Saved: pics/q2/frame_0025.jpg
Saved: pics/q2/frame_0026.jpg
Saved: pics/q2/frame_0027.jpg
Saved: pics/q2/frame_0028.jpg
Saved: pics/q2/frame_0029.jpg
Saved: pics/q2/frame_0030.jpg
Saved: pics/q2/frame_0031.jpg
Saved: pics/q2/frame_0032.jpg
Saved: pic

KeyboardInterrupt: 

### Q3.

In [40]:
def merge_images(foreground, background):
    if foreground is None or background is None:
        return None

    if foreground.shape[:2] != background.shape[:2]:
        background = cv2.resize(background, (foreground.shape[1], foreground.shape[0]))

    green_lower = np.array([0, 100, 0])
    green_upper = np.array([100, 255, 100])

    mask = cv2.inRange(foreground, green_lower, green_upper)
    mask_inv = cv2.bitwise_not(mask)
    
    foreground_masked = cv2.bitwise_and(foreground, foreground, mask=mask_inv)
    background_masked = cv2.bitwise_and(background, background, mask=mask)

    result = cv2.add(background_masked, foreground_masked)
    return result


In [41]:
def merge_videos(video_path1, video_path2, output_path):
    
    output_folder = os.path.dirname(output_path)
    os.makedirs(output_folder, exist_ok=True)

    
    video1 = cv2.VideoCapture(video_path1)
    video2 = cv2.VideoCapture(video_path2)

    
    frame_rate = video1.get(cv2.CAP_PROP_FPS)

    
    success1, frame1 = video1.read()
    success2, frame2 = video2.read()

    
    if not success1 or not success2:
        print("Error reading video frames.")
        return

    height, width, layers = frame1.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    
    video_out = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))

    
    while True:
        merged_frame = merge_images(frame1, frame2)
        if merged_frame is None:
            break

        video_out.write(merged_frame)

       
        success1, frame1 = video1.read()
        success2, frame2 = video2.read()

    
    video1.release()
    video2.release()
    video_out.release()
    cv2.destroyAllWindows()



In [43]:
merge_videos("tw.mp4","earth.mp4", "video/q3/q3_alternate_merge.mp4")